In [1]:
!tree models

models
└── model1
    ├── assets
    ├── keras_metadata.pb
    ├── saved_model.pb
    └── variables
        ├── variables.data-00000-of-00001
        └── variables.index

3 directories, 4 files


In [2]:
import tensorflow as tf
import PIL
import pathlib
import numpy as np

### Load and test model

In [31]:
model = tf.keras.models.load_model('models/model1')
class_names = ["masza", "rysia"]

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 180, 180, 3)       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 90, 90, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  

In [33]:
cats_nonseen = pathlib.Path("./dataset/mycats_nontraning/")
masza_nonseen = list(koty_nowe.glob('masza*'))
rysia_nonseen = list(koty_nowe.glob('rysia*'))
print(rysia_nowe, masza_nowe)

[PosixPath('koty_nowe/rysia2.jpg'), PosixPath('koty_nowe/rysia1.jpg')] [PosixPath('koty_nowe/masza2.jpg'), PosixPath('koty_nowe/masza1.jpg')]


In [35]:
newsize = (300, 300)
PIL.Image.open(str(rysia_nonseen[0])).resize(newsize)

IndexError: list index out of range

### Predict

In [6]:
img_height = 180
img_width = 180

In [20]:
img = tf.keras.utils.load_img(
    str(rysia_nowe[1]), target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

1/1 [==============================] - 0s 19ms/step


In [21]:
print(list(model.signatures.keys()))

['serving_default']


In [22]:
class_names = ["masza", "rysia"]

In [23]:
print(
    "this picture presents `{}` with a `{:.2f}` percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

this picture presents `rysia` with a `57.99` percent confidence.
